### Pivot table

+----+------+-------+
| ID | Type | Value |
+----+------+-------+
| 1  | A    | 10    |
| 1  | B    | 20    |
| 2  | A    | 30    |
| 2  | B    | 40    |
+----+------+-------+
to 
+----+---------+---------+
| ID | A_Value | B_Value |
+----+---------+---------+
| 1  | 10      | 20      |
| 2  | 30      | 40      |
+----+---------+---------+



In [ ]:
# need Max here since 
SELECT
  ID,
  MAX(CASE WHEN Type = 'A' THEN Value END) AS A_Value,
  MAX(CASE WHEN Type = 'B' THEN Value END) AS B_Value
FROM
  your_table_name
GROUP BY
  ID;


In [ ]:
WITH t1 as (
SELECT candidate_id,
      MAX(CASE WHEN skill = 'Python' THEN 'yes' END) as python,
      MAX(CASE WHEN skill = 'Tableau' THEN 'yes' END) as tableau,
      MAX(CASE WHEN skill = 'PostgreSQL' THEN 'yes' END) as postgresql
FROM candidates
GROUP BY candidate_id)

SELECT candidate_id
FROM t1
WHERE python is not NULL
AND tableau is not NULL
AND postgresql is not NULL
ORDER BY candidate_id ASC;

## Date Calculation in PostgreSql

In [ ]:
SELECT 
	user_id, 
    MAX(post_date::DATE) - MIN(post_date::DATE) AS days_between
FROM posts
WHERE DATE_PART('year', post_date::DATE) = 2021 
GROUP BY user_id
HAVING COUNT(post_id)>1;

## Find Duplicates with Group By

In [ ]:
WITH job_count_cte AS (
  SELECT 
    company_id, 
    title, 
    description, 
    COUNT(job_id) AS job_count
  FROM job_listings
  GROUP BY company_id, title, description
)

SELECT COUNT(DISTINCT company_id) AS duplicate_companies
FROM job_count_cte
WHERE job_count > 1;

## Concat Number and symbol or character

In [ ]:
SELECT 
  manufacturer, 
  CONCAT('$', ROUND(SUM(total_sales) / 1000000), ' million') AS sales_mil
FROM pharmacy_sales 
GROUP BY manufacturer;